# MVBC Alg

## Functions

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import *
import csv
from sklearn.cluster import KMeans
from skimage.restoration import denoise_wavelet 
import pywt
import time
from math import radians, cos, sin, asin, sqrt

def normalization (x):
    return ((x-min(x))/(max(x)-min(x)))



array([0.        , 0.33333333, 0.66666667, 1.        ])